In [17]:
%cd /content/drive/MyDrive/ColabProjects

/content/drive/MyDrive/CollabProjects


In [15]:
!git clone https://github.com/OdyAsh/nlp-image-captioning.git

Cloning into 'nlp-image-captioning'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 4), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (22/22), 8.02 KiB | 29.00 KiB/s, done.


In [33]:
%cd /content/drive/MyDrive/ColabProjects/nlp-image-captioning

/content/drive/MyDrive/ColabProjects/nlp-image-captioning


In [35]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '_tflow_select==2.3.0' (from line 1 of requirements.txt)


In [28]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/ColabProjects/nlp-image-captioning/convert_envy_to_reqtxt')

In [31]:
from convert_envy_to_reqtxt import convert_environment_yaml_to_requirements_txt
convert_environment_yaml_to_requirements_txt()

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
print(requests)

<module 'requests' from 'd:\\CS\\projects\\nlp-image-captioning\\.conda\\lib\\site-packages\\requests\\__init__.py'>
